In [1]:
from Zaif_api import Zaif_api
import yaml
import datetime
import time

In [2]:
path = "../api_config.yaml"
with open(path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
budget = config['budget']
api_key = config['api_key']
api_secret = config['api_secret']

In [3]:
api = Zaif_api.zaif(api_key, api_secret)

In [ ]:
print("Job starting at {}".format(datetime.datetime.now()))

##### Cancel all orders #####
api.cancel_all_orders()
print("All orders cancelled")

##### Get decimal limit #####
currency_info = api.get_pair_info()
decimal_length = len(str(currency_info['item_unit_step']))-2

##### Place order at the highest bid price #####
highest_bid = api.get_highest_bid()
bid_price = highest_bid + 100 -5000
bid_amount = round(budget/bid_price, decimal_length)
trade_result = api.trade(price=bid_price, amount=bid_amount)
print("Bid placed at {} for {} ETH".format(bid_price, bid_amount))

##### Get my balance ####
balance_amount, balance_available = api.my_balance("jpy")


##### Repeat until my order gets fulfilled #####
while balance_amount != balance_available:
    
    # Keep my bid the highest on the board
    if bid_price != api.get_highest_bid():
        
        # Get outstanding bid amount
        active_order = api.active_orders()
        try:
            outstanding_amount = list(active_order['return'].values())[0]['amount']
        except:
            outstanding_amount = 0
        
        # Re-bid
        bid_price = api.get_highest_bid() + 100 - 5000
        api.cancel_all_orders()
        api.trade(price=bid_price, amount=outstanding_amount)
        print("Bid placed at {} for {} ETH".format(bid_price, outstanding_amount))
        
    time.sleep(1)
    balance_amount, balance_available = api.my_balance("jpy")
    print("    amount = {}, available = {}".format(balance_amount, balance_available))
    
api.cancel_all_orders()
print("Order fulfilled")

Job starting at 2021-09-14 14:27:14.484661
All orders cancelled
Bid placed at 359430.0 for 0.0011 ETH
Bid placed at 359435.0 for 0.0011 ETH
    amount = 383126.88701291, available = 383522.26551291
Bid placed at 359450.0 for 0.0011 ETH
    amount = 383126.87051291, available = 383522.26551291
Bid placed at 359465.0 for 0.0011 ETH
    amount = 383126.85401291, available = 383522.26551291
Bid placed at 359485.0 for 0.0011 ETH
    amount = 383126.83201291, available = 383522.26551291
Bid placed at 359500.0 for 0.0011 ETH
    amount = 383126.81551291, available = 383522.26551291
Bid placed at 359490.0 for 0.0011 ETH
    amount = 383126.82651291, available = 383522.26551291
Bid placed at 359505.0 for 0.0011 ETH
    amount = 383126.81001291, available = 383522.26551291
Bid placed at 359525.0 for 0.0011 ETH
    amount = 383126.78801291, available = 383522.26551291


# テスト用